In [ ]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import torch
from transformers import ViTForImageClassification, AutoImageProcessor
from datasets import load_dataset
from torch.profiler import profile, ProfilerActivity, record_function
import cProfile
import pstats
import io
from collections import defaultdict

from faeyon.io import load
from faeyon import X

repo = "google/vit-base-patch16-224"

In [3]:
hf_model = ViTForImageClassification.from_pretrained(repo)
hf_model.eval()
hf_model.cuda()
pass

In [4]:
model = load("vit/vit-base-patch16-224", cache=True)
model.eval()
model.cuda()
pass

In [5]:
image_processor = AutoImageProcessor.from_pretrained(repo)
imagenet = load_dataset("ILSVRC/imagenet-1k", trust_remote_code=True)
inputs = image_processor(
    images=imagenet["train"][0]["image"],
    return_tensors="np"
)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'ILSVRC/imagenet-1k' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Resolving data files:   0%|          | 0/294 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/294 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/267 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [6]:
img = torch.tensor(inputs["pixel_values"]).cuda()

In [25]:
%%timeit

hf_y = hf_model(img)


3.28 ms ± 1.09 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%%timeit
y = model(img)

3.4 ms ± 1.07 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
hf_y = hf_model(img)
y = model(img)
torch.allclose(hf_y.logits,  y)

True

## Profiling

In [ ]:
# Analyze and compare
def analyze_profile(pr, ndata = sorted([(stat.code, stat.callcount, stat.inlinetime) for stat in pr.getstats()], key=lambda x: x[1], reverse=True)ame):
    """Analyze a profile and return statistics"""
    stats = pr.getstats()
    total_time = sum(stat.totaltime for stat in stats)
    
    # Group by module
    by_module = defaultdict(lambda: {"time": 0.0, "calls": 0, "functions": []})
    
    for stat in stats:
        if stat.code and not isinstance(stat.code, str):
            filename = stat.code.co_filename
            func_name = stat.code.co_name
            module = filename.split('/')[-1] if '/' in filename else filename.split('\\')[-1]
            
            by_module[module]["time"] += stat.totaltime
            by_module[module]["calls"] += stat.callcount
            if stat.totaltime > 0.0001:  # Only track functions taking > 0.1ms
                by_module[module]["functions"].append({
                    "name": func_name,
                    "time": stat.totaltime,
                    "calls": stat.callcount
                })
    
    # Sort functions by time
    for module in by_module:
        by_module[module]["functions"].sort(key=lambda x: x["time"], reverse=True)
    
    return {
        "total_time": total_time,
        "by_module": dict(by_module)
    }

_ = model(img)
torch.cuda.synchronize()

# Profile Hugging Face model
with cProfile.Profile() as pr:
    y = model(img)

pr.print_stats(sort=0)

         20152 function calls (17155 primitive calls) in 0.009 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
4301/3148    0.001    0.000    0.002    0.000 {built-in method builtins.isinstance}
     1628    0.000    0.000    0.000    0.000 <frozen abc>:117(__instancecheck__)
     1628    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck}
     1153    0.000    0.000    0.001    0.000 x.py:160(_meta_instancecheck)
     1153    0.000    0.000    0.000    0.000 {function _meta_instancecheck at 0x7f8b6036f100}
      691    0.000    0.000    0.001    0.000 spells.py:133(<genexpr>)
  629/461    0.000    0.000    0.000    0.000 {built-in method builtins.len}
      472    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
      396    0.000    0.000    0.000    0.000 module.py:1949(__getattr__)
   361/79    0.000    0.000    0.006    0.000 spells.py:155(<genexpr>)
      343    0.000    0.000  

In [ ]:
[1, 2, 3] >> X[X[0]]

2

In [ ]:
data = sorted([(stat.code, stat.callcount, stat.inlinetime) for stat in pr.getstats()], key=lambda x: x[1], reverse=True)
data

[('<built-in method builtins.isinstance>', 4301, 0.000592201),
 (<code object __instancecheck__ at 0x7f8f7cb17050, file "<frozen abc>", line 117>,
  1628,
  0.00019933),
 ('<built-in method _abc._abc_instancecheck>', 1628, 0.00023474400000000003),
 (<code object _meta_instancecheck at 0x7f8b60731920, file "/home/gerges/Documents/faeyon-ml/faeyon/faeyon/magic/x.py", line 160>,
  1153,
  0.00041306400000000004),
 ('<function _meta_instancecheck at 0x7f8b6036f100>',
  1153,
  0.00011150600000000001),
 (<code object <genexpr> at 0x7f8b60328390, file "/home/gerges/Documents/faeyon-ml/faeyon/faeyon/magic/spells.py", line 133>,
  691,
  0.000118844),
 ('<built-in method builtins.len>', 629, 8.209100000000001e-05),
 ("<method 'append' of 'list' objects>", 472, 4.6196e-05),
 (<code object __getattr__ at 0x564d02b9ff10, file "/home/gerges/Documents/faeyon-ml/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py", line 1949>,
  396,
  0.00011030100000000001),
 (<code object <genexpr> at 0

In [ ]:
spells = [stat for stat in stats if not isinstance(stat.code, str) and "spells" in stat.code.co_filename]
faek = [stat for stat in stats if not isinstance(stat.code, str) and "faek" in stat.code.co_filename]
spells = sorted(spells, key=lambda x: x.inlinetime * x.reccallcount, reverse=True)

NameError: name 'stats' is not defined

In [ ]:
spells[0].calls#.code.co_firstlineno

[_lsprof.profiler_subentry(code=<code object using at 0x7fcc4de3d2c0, file "/home/gerges/Documents/faeyon-ml/faeyon/faeyon/magic/spells.py", line 225>, callcount=62, reccallcount=12, totaltime=0.005316717, inlinetime=1.3765000000000001e-05),
 _lsprof.profiler_subentry(code='<built-in method builtins.getattr>', callcount=102, reccallcount=0, totaltime=2.6391000000000002e-05, inlinetime=2.6391000000000002e-05),
 _lsprof.profiler_subentry(code='<built-in method builtins.isinstance>', callcount=988, reccallcount=0, totaltime=0.000732716, inlinetime=0.000178622),
 _lsprof.profiler_subentry(code=<code object _meta_iter at 0x7fcc4da5e870, file "/home/gerges/Documents/faeyon-ml/faeyon/faeyon/magic/x.py", line 134>, callcount=134, reccallcount=0, totaltime=3.3207e-05, inlinetime=2.2293e-05),
 _lsprof.profiler_subentry(code="<method 'items' of 'dict' objects>", callcount=102, reccallcount=0, totaltime=1.0708000000000001e-05, inlinetime=1.0708000000000001e-05),
 _lsprof.profiler_subentry(code=<co

In [ ]:
# Find the top-level function (usually the model's __call__ or forward)
top_level = max(stats, key=lambda s: s.totaltime if s.code else 0)
total_time = top_level.totaltime  # ✅ This is the actual total time

In [ ]:
from collections import Counter

lines  = []
for s in spells:
    lines.append(s.code.co_qualname)

Counter(lines)


Counter({'conjure': 1,
         '_CallableStrategy.__call__': 1,
         'Delayable.using': 1,
         'Parallels.__getitem__': 1,
         'A.using.<locals>.<genexpr>': 1,
         'A.using': 1,
         'Serials._resolve': 1,
         'Op._resolve': 1,
         'A.__rrshift__': 1,
         'Delayable.__rrshift__': 1,
         'Op.__init__': 1,
         'Serials.__init__': 1,
         'Parallels.__init__': 1,
         '_OpBase._binary_op': 1,
         '_XStrategy.__init__': 1,
         '_Fanout.__init__': 1,
         '_NoopStrategy.__call__': 1,
         'Parallels._resolve': 1,
         'Parallels.__len__': 1,
         '_new_instance': 1,
         'A.__init__.<locals>.<genexpr>': 1,
         'Delayable.__rshift__': 1,
         'Delayable.__lshift__': 1,
         'ContainerBase.copy': 1,
         'Delayable.__new__': 1,
         'Delayable.if_': 1,
         'ContainerBase.__init__': 1,
         'A.is_resolved': 1,
         'ContainerBase.__matmul__': 1,
         'ContainerBase.__rrs